In [ ]:
###Josephine Esposito - A00827145     27/08/2021

#Actividad en clase: cajas
---

Realizar una simulación en colab con agentes. En esta simulación el objetivo es realizar pilas de máximo cinco cajas. Las cajas, al empezar la simulación, deberán nacer en modo random en las celdas según el número preestablecido. De la misma manera los agentes, al empezar la simulación, deberán de nacer en modo random en las celdas según el número preestablecido. Dos agentes no pueden estar en la misma celda al mismo tiempo nunca. Al nacer, dos cajas, o más, no pueden estar en la misma celda.

La simulación se concluye cuando el tiempo preestablecido llega a cero o cando ya no hay cajas que se puedan poner en pila.

# Imports

In [4]:
!pip install mesa

In [5]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd
import random

import time
import datetime

# Crear el modelo

In [60]:
def obt_habitacion(modelo):
  habitacion = np.zeros((modelo.grid.width, modelo.grid.height))

  for celda in modelo.grid.coord_iter():
    contenido_celda, x, y = celda
    for contenido in contenido_celda:
      if isinstance(contenido, Robot):
        habitacion[x][y] = 6
      else:
        habitacion[x][y] = contenido.estado
  return habitacion

#==============================================================================
class Robot(Agent):
  def __init__(self, id_unico, modelo):
    super().__init__(id_unico, modelo)
    self.nueva_posicion = None
    self.movimientos = 0
    self.CARRY = 0 # var that tells me if I'm caarieng the box with me
    # function that allows me to position a box
    # function that searchs the nearest and/or biggest pile of boxes

  def step(self):
    vecinos = self.model.grid.get_neighbors(self.pos, moore=True, include_center=True)

    # We define the next state
    for vecino in vecinos:
      if isinstance(vecino, Caja) and vecino.pos == self.pos:
        if self.CARRY == 1:
          # check where i can drop box
          vecino.siguiente_estado = vecino.estado
          if vecino.siguiente_estado >= vecino.SING and vecino.siguiente_estado < vecino.MAX:
            self.CARRY = 0
            vecino.siguiente_estado = vecino.estado + 1
          else:
            vecindario = self.model.grid.get_neighborhood(self.pos, moore=True, include_center = False)
            nueva_posicion = self.random.choice(vecindario)
            self.nueva_posicion = nueva_posicion
          break
        elif self.CARRY == 0:
          # check where i can pick box
          vecino.siguiente_estado = vecino.estado
          if vecino.siguiente_estado == vecino.SING:
            self.CARRY = 1
            vecino.siguiente_estado = vecino.VAC
          else:
            vecindario = self.model.grid.get_neighborhood(self.pos, moore=True, include_center = False)
            nueva_posicion = self.random.choice(vecindario)
            self.nueva_posicion = nueva_posicion
          break
    
  
  def advance(self):
    # We actualize the state of the floor
    vecinos = self.model.grid.get_neighbors(self.pos, moore = False, include_center = True)

    for vecino in vecinos:
      if isinstance(vecino, Caja) and vecino.pos == self.pos:
        vecino.estado = vecino.siguiente_estado
        break
    # We count the number of movements
    if self.pos != self.nueva_posicion:
      self.movimientos = self.movimientos + 1
    
    # We move the robot to its new position
    self.model.grid.move_agent(self, self.nueva_posicion)

#==============================================================================
class Caja(Agent):
  VAC = 0     # no box
  SING = 1    # one box
  MAX = 5     # full pile of boxes

  # We initialize the floor with no boxes
  def __init__(self, pos, modelo, estado = VAC):
    super().__init__(pos, modelo)
    self.x, self.y = pos
    self.estado = estado
    self.siguiente_estado = None
  
  def add(self):
    self.estado = self.estado + 1

#==============================================================================
class Habitacion(Model):
  # We initialize the floor with all the elements
  def __init__(self, m, n, num_agentes, por_num_cajas):
    self.num_agentes = num_agentes
    self.por_num_cajas = por_num_cajas
    self.por_celdas_limpias = 1 - por_num_cajas
    self.grid = MultiGrid(m, n, True)
    self.schedule = SimultaneousActivation(self)

    # We position the boxes in random positions
    num_cajas = int( ((m * n) - (5 + 1)) * por_num_cajas ) # = celdas_sucias
    lista_celdas_vacias = list(self.grid.empties)
    for celdas in range(num_cajas):
      celda_vacia = random.choice(lista_celdas_vacias)
      caja = Caja(celda_vacia, self)
      caja.estado = caja.SING
      self.grid.place_agent(caja, celda_vacia)
      self.schedule.add(caja)
      lista_celdas_vacias.remove(celda_vacia)

    # We position the "empty boxes"
    lista_celdas_vacias = list(self.grid.empties)
    for celdas in lista_celdas_vacias:
      caja = Caja(celdas, self)
      self.grid.place_agent(caja, celdas)
      self.schedule.add(caja)
    
    # We position the robots
    for i in range(num_agentes):
      robot = Robot(i, self)
      self.grid.place_agent(robot, (1,1)) # this need to be randomized!!!
      self.schedule.add(robot)
    
    # Data Collector
    self.colector_datos = DataCollector(model_reporters = {'Habitacion': obt_habitacion},
                                        agent_reporters = {'Movimientos': lambda a: getattr(a, 'movimientos', None)})
    
  def step(self):
    self.colector_datos.collect(self)
    self.schedule.step()


# Correr el modelo

In [65]:
# Datos de la habitación
M = 20
N = 20

# Número de Robots
NUM_ROBOTS = 6

# Porcentaje de celdas inicialmente llenas
PORCENTAJE_NUMERO_CAJAS = 0.2

# Tiempo máximo de ejecución en segundos
TIEMPO_MAXIMO_EJECUCION = 0.66


#==============================================================================
start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds = TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_ROBOTS, PORCENTAJE_NUMERO_CAJAS)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION):
  modelo.step()

# Imprimimos el tiempo que le tomó correr al modelo
tiempo_ejecucion = str(datetime.timedelta(seconds = (time.time() - start_time)))


# Visualización

In [66]:
todas_habitaciones = modelo.colector_datos.get_model_vars_dataframe()

In [68]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap = 'Greys')

def animate(i):
  patch.set_data(todas_habitaciones.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames = len(todas_habitaciones))


In [69]:
anim

# Informe

In [70]:
movimientos = modelo.colector_datos.get_agent_vars_dataframe()

print('Tiempo necesario hasta que todas las cajas estén organizadas:', tiempo_ejecucion, '/', tiempo_inicio)
print('Número de movimientos realizados por todos los agentes:', movimientos.tail()['Movimientos'].sum())

Tiempo necesario hasta que todas las cajas estén organizadas: 0:00:00.660251 / 0:00:00.660000
Número de movimientos realizados por todos los agentes: 1980.0
